# embryo_binary_segmentation train_mouse_embryo

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchio as tio

In [2]:
import embryo_binary_segmentation._unet_smaller as unet_smaller
from embryo_binary_segmentation._data_load import upload_data
from embryo_binary_segmentation._losses import dice_loss, focal_loss
from embryo_binary_segmentation._config import DATA_PARAMS, FINE_TUNING, TRAINING_PARAMS
from embryo_binary_segmentation._train_functions import train

/home/gesaloof/code/IO/IO/IO.py:24: UserWarning: KLB library is not installed
  warnings.warn('KLB library is not installed')


# Load parameters

In [3]:
loss, learning_rate, batch_size, epochs, save_model_path, fine_tuning, save_each = TRAINING_PARAMS.values()

In [4]:
data_path, binarize, target_size, patch_size, augmentations = DATA_PARAMS.values()

train_folder = f"{data_path}Train/"
val_folder = f"{data_path}Val/"

In [5]:
upload_model_path, old_steps = FINE_TUNING.values()

In [6]:
if loss == 'bce':
    loss_fn = nn.BCELoss()
elif loss == 'dice':
    loss_fn = dice_loss
elif loss == 'focal':
    loss_fn = focal_loss  

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device, '\n')

cuda 



# Upload Data

In [8]:
patch_size

[32, 256, 256]

In [9]:
train_dataset = upload_data(train_folder, 'train', binarize, patch_size)
print("Train data is loaded \n")

/home/gesaloof/data/data_polina/Train/SEG_seeds_from_previous_binary
/home/gesaloof/data/data_polina/Train/FUSE
/home/gesaloof/data/data_polina/Train/SEG_seeds_from_prev_cropped_binary
/home/gesaloof/data/data_polina/Train/FUSE_raw_cropped
Train data is loaded 



In [10]:
train_dataset[0]['image'][tio.DATA].shape

torch.Size([1, 32, 256, 256])

In [11]:
val_dataset = upload_data(val_folder, 'val', binarize, patch_size)
print("Validation data is loaded \n")

/home/gesaloof/data/data_polina/Val/SEG_seeds_from_previous_binary
/home/gesaloof/data/data_polina/Val/FUSE
/home/gesaloof/data/data_polina/Val/SEG_seeds_from_prev_cropped_binary
/home/gesaloof/data/data_polina/Val/FUSE_raw_cropped
Validation data is loaded 



In [12]:
if augmentations:
    spatial = tio.OneOf({
        tio.RandomElasticDeformation(num_control_points=(6, 6, 8), locked_borders=2, max_displacement=(16, 16, 2)): 0.1,
        tio.RandomAffine(scales=(1, 1.05), degrees=5): 0.2,
        tio.RandomFlip(axes=('LR',)): 0.1,
        tio.RandomGhosting(): 0.2,
        tio.RandomBiasField():0.1,
        tio.RandomNoise(): 0.1
                        },
        p=0.8,)

    subjects_dataset = tio.SubjectsDataset(train_dataset, transform=spatial)

    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
    train_loader = DataLoader(subjects_dataset, batch_size=batch_size, shuffle=True)

else:
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)


print("Dataloaders are created \n")

Dataloaders are created 



# Train

In [21]:
unet_small = unet_smaller.UNet()
unet_small = unet_small.to(device)
optim = torch.optim.SGD(unet_small.parameters(), lr=learning_rate)

In [18]:
for param in (device, optim, loss_fn, 5, train_loader, val_loader, save_model_path, save_each, upload_model_path, old_steps):
    print(param)

cuda
SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    fused: None
    lr: 0.0001
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
)
BCELoss()
5
/home/gesaloof/data/data_polina/Models/Test
True
/home/gesaloof/data/data_polina/Models/Test/best.model
2


In [22]:
min_val_loss, best_epoch = train(unet_small, device, optim, loss_fn, 200, train_loader, val_loader, save_model_path, save_each, upload_model_path, old_steps)

Loaded model weights from /home/gesaloof/data/data_polina/Models/Test/best.model
* Epoch 3/202


Training: 100%|██████████████████████████████████████████████████████████████████████████████████████████| 51/51 [00:44<00:00,  1.14it/s]


Train loss: 0.491148
New best model saved with loss 0.4799889922142029
* Epoch 4/202


Training: 100%|██████████████████████████████████████████████████████████████████████████████████████████| 51/51 [00:44<00:00,  1.15it/s]


Train loss: 0.485028
New best model saved with loss 0.47356390953063965
* Epoch 5/202


...

Training: 100%|██████████████████████████████████████████████████████████████████████████████████████████| 51/51 [00:49<00:00,  1.02it/s]


Train loss: 0.267011
New best model saved with loss 0.24162234365940094
* Epoch 197/202


Training: 100%|██████████████████████████████████████████████████████████████████████████████████████████| 51/51 [00:49<00:00,  1.02it/s]


Train loss: 0.263813
New best model saved with loss 0.24147507548332214
* Epoch 198/202


Training:  39%|███████████████████████████████████▎                                                      | 20/51 [00:19<00:27,  1.12it/s]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [23]:
print(f"Best epoch: {best_epoch}, Min Validation Loss: {min_val_loss}")

Best epoch: 201, Min Validation Loss: 0.23914127051830292


In [16]:
del unet_small
torch.cuda.empty_cache()

In [19]:
train??

Signature:
train(
    model,
    device,
    opt,
    loss_fn,
    epochs,
    data_tr,
    data_val,
    save_path,
    save_each=True,
    upload_path=None,
    old_steps=0,
)
Docstring: <no docstring>
Source:   
def train(model, device, opt, loss_fn, epochs, data_tr, data_val, save_path, save_each=True, upload_path=None, old_steps=0):
    min_val_loss = float('inf')
    best_epoch = 0
    model_save_path = f"{save_path}/best.model"
    csv_save_path  = f"{save_path}/training_data.csv"

    if upload_path != None:
        if os.path.exists(upload_path):
            model.load_state_dict(torch.load(upload_path))
            print("Loaded model weights from", upload_path)
    
    for epoch_1 in range(epochs):
        epoch = epoch_1 + old_steps
        tic = time()
        print('* Epoch %d/%d' % (epoch+1, epochs+old_steps))

        avg_loss = 0
        model.train() 
        for batch in tqdm(data_tr, desc="Training", leave=True):
            X_batch, Y_batch = batch['image'][tio.DA